In [64]:
import ee
import geemap as emap

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhYI3HNXK0WS4Xal4rzUG4wYzozkPWFVo-_uGobCziTUsW1bs6Gz_8

Successfully saved authorization token.


In [65]:
Map=emap.Map(center=[11.94,108.45],zoom=7)
Map

Map(center=[11.94, 108.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

# Masking Landsat 8 

- **Using median reducer**

In [66]:
# Select Landsat 8 level-1 TOA Collection
LS8= ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
# Define the study boundary
LD=ee.FeatureCollection("users/miketu72/2021_LD_test_area")

LS8_2020=LS8.filterDate("2020-01-01","2021-01-10").filterBounds(LD).filterMetadata("CLOUD_COVER","less_than",50).median()
# Visualization pramaters 
vis={"bands":["B4","B3","B2"],min:-0.2, max:0.9}

Map.addLayer(LS8_2020,vis_params,"LS Without Mask")
Map

Map(bottom=15589.0, center=[11.94, 108.45], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

- **Masked all water areas**

In [67]:
# // Load or import the Hansen et al. forest change dataset.
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')

# // Select the land/water mask.
datamask = hansenImage.select('datamask')

# Map.addLayer(datamask,{},"Mask")

# // Create a binary mask.
mask = datamask.eq(1)

# // Update the composite mask with the water mask.
maskedComposite = LS8_2020.updateMask(mask)
Map.addLayer(maskedComposite, vis_params, 'masked')
Map

Map(bottom=15589.0, center=[11.94, 108.45], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

- **Visualizing multiple layers**

Visualize the natural color Landsat within the study area while all water will be blue

In [68]:
# Define the visualization parameters
vis={"bands":["B4","B3","B2"],"min":-0.2, "max":0.9}
# Convert water will be one and all other will be zeros
water=mask.eq(0) 
# Mask out landmass
water_mask=mask.mask(water) # This mask is similar to updateMask()
# Build a ImageColleciton with visualized parameters
LS8_vis=LS8_2020.visualize(**vis)

water_vis=water_mask.visualize(**{"palette": "000044"})

# Mosaic two images together by putting them in a collection
img_mosaic=ee.ImageCollection([LS8_vis,water_vis]).mosaic()

Map.addLayer(img_mosaic,{},"Mosaic")
Map

Map(bottom=15589.0, center=[11.94, 108.45], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

# Quality Mosaic with NDVI

In [83]:
LS8_2020=LS8.filterDate("2020-01-01","2021-01-10").filterBounds(LD).filterMetadata("CLOUD_COVER","less_than",50)
# Write a function to calculate NDVI

def ndvi(image):
    NDVI=image.expression("(NIR-RED)/(NIR+RED)", 
                          {"NIR":image.select("B5"), 
                           "RED":image.select("B4")}).rename("NDVI")
    return image.addBands(NDVI)
# Apply map function to the defined function

LS8_2020NDVI=LS8_2020.map(ndvi)
# Select the greenest pixels by qualityMosaic()

greenest=LS8_2020NDVI.qualityMosaic("NDVI")
# Display it
vis={"bands":["B4","B3","B2"],"min":-0.2, "max":0.5}

Map.addLayer(greenest,vis,"Greenest") # We see that water becomes white because of quality mosaic effects.
Map

Map(bottom=61078.0, center=[12.956382587313202, 108.43780517578126], controls=(WidgetControl(options=['positio…